In [7]:
import pandas as pd
import numpy as np

# Path to your CSV file
csv_file_path = 'white_wine.csv'  # Update with your actual path

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Process the 'cov_value' and 'PINAW_value' columns
df['cov_value'] = pd.to_numeric(df['cov_value'], errors='coerce')
df['PINAW_value'] = pd.to_numeric(df['PINAW_value'], errors='coerce')

# Drop rows with NaN values in 'cov_value' or 'PINAW_value'
df = df.dropna(subset=['cov_value', 'PINAW_value'])

# Add a 'coverage_level' column based on the order of results
df['result_index'] = df.groupby(['add_remove_every_n_epoch', 'proto_split_density_threshold',
                                 'proto_remove_density_threshold', 'repulsion_loss_margin',
                                 'seed']).cumcount()

# Map result_index to coverage levels
coverage_level_mapping = {0: 90, 1: 50, 2: 10}
df['coverage_level'] = df['result_index'].map(coverage_level_mapping)

# Define hyperparameter columns
hyperparameter_cols = ['add_remove_every_n_epoch', 'proto_split_density_threshold',
                       'proto_remove_density_threshold', 'repulsion_loss_margin']

group_cols = hyperparameter_cols + ['coverage_level']

# Compute the average and standard deviation for each hyperparameter setting across seeds
grouped = df.groupby(group_cols).agg({
    'cov_value': ['mean', 'std'],
    'PINAW_value': ['mean', 'std'],
}).reset_index()

# Flatten the MultiIndex columns
grouped.columns = ['_'.join(col).strip('_') for col in grouped.columns.values]

# Now, define the function to find the best hyperparameters
def find_best_hyperparameters(grouped_df, coverage_target, tolerance=5):
    # Calculate the acceptable coverage range
    min_acceptable_coverage = (coverage_target - tolerance) / 100   
    max_acceptable_coverage = (coverage_target + tolerance) / 100

    # Filter for the desired coverage level
    coverage_df = grouped_df[grouped_df['coverage_level'] == coverage_target]

    # Filter hyperparameters within the acceptable coverage range
    acceptable_df = coverage_df[
        (coverage_df['cov_value_mean'] >= min_acceptable_coverage) &
        (coverage_df['cov_value_mean'] <= max_acceptable_coverage)
    ]

    if acceptable_df.empty:
        print(f"No hyperparameter settings found within {tolerance}% tolerance for {coverage_target}% coverage.")
        return None

    # Find the hyperparameter setting(s) with the minimum PINAW_value_mean
    min_pinaw = acceptable_df['PINAW_value_mean'].min()
    best_settings = acceptable_df[acceptable_df['PINAW_value_mean'] == min_pinaw]

    return best_settings

# Find best hyperparameters for each coverage level
best_settings_90 = find_best_hyperparameters(grouped, 90, tolerance=5)
best_settings_50 = find_best_hyperparameters(grouped, 50, tolerance=5)
best_settings_10 = find_best_hyperparameters(grouped, 10, tolerance=5)

# Print the best hyperparameter settings
print("Best hyperparameters for 90% coverage level:")
print(best_settings_90[hyperparameter_cols + ['cov_value_mean', 'PINAW_value_mean']])

print("\nBest hyperparameters for 50% coverage level:")
print(best_settings_50[hyperparameter_cols + ['cov_value_mean', 'PINAW_value_mean']])

print("\nBest hyperparameters for 10% coverage level:")
print(best_settings_10[hyperparameter_cols + ['cov_value_mean', 'PINAW_value_mean']])

# Analyze the influence of each hyperparameter on the results
def analyze_hyperparameter_influence(grouped_df):
    # Ensure hyperparameters are numeric
    analysis_df = grouped_df.copy()
    for col in hyperparameter_cols:
        analysis_df[col] = pd.to_numeric(analysis_df[col], errors='coerce')

    # Drop any rows with NaN hyperparameters
    analysis_df = analysis_df.dropna(subset=hyperparameter_cols)

    # Compute the correlation matrix
    corr_matrix = analysis_df[hyperparameter_cols + ['cov_value_mean', 'PINAW_value_mean']].corr()

    print("\nCorrelation matrix between hyperparameters and PINAW_value_mean:")
    print(corr_matrix['PINAW_value_mean'])

    print("\nCorrelation matrix between hyperparameters and cov_value_mean:")
    print(corr_matrix['cov_value_mean'])

# Call the analysis function
analyze_hyperparameter_influence(grouped)

Best hyperparameters for 90% coverage level:
     add_remove_every_n_epoch  proto_split_density_threshold  \
167                       100                          0.002   

     proto_remove_density_threshold  repulsion_loss_margin  cov_value_mean  \
167                          0.0002                   0.01        0.904923   

     PINAW_value_mean  
167          0.411192  

Best hyperparameters for 50% coverage level:
     add_remove_every_n_epoch  proto_split_density_threshold  \
154                       100                          0.002   

     proto_remove_density_threshold  repulsion_loss_margin  cov_value_mean  \
154                          0.0001                   0.01        0.490771   

     PINAW_value_mean  
154          0.066454  

Best hyperparameters for 10% coverage level:
     add_remove_every_n_epoch  proto_split_density_threshold  \
153                       100                          0.002   

     proto_remove_density_threshold  repulsion_loss_margin  cov_va

In [10]:
grouped["cov_value"]

0      0.116505
1      0.544661
2      0.913592
3      0.137865
4      0.524274
         ...   
211    0.526213
212    0.916505
213    0.108736
214    0.545630
215    0.919419
Name: cov_value, Length: 216, dtype: float64